# Simulation of Mass Action Kinetics
This is an example notebook for interactive simulation of a user-defined chemical reaction network. 

## Initializing a Chemical Reaction Network
Below is an example enzymatic reaction modelled with mass action kinetics. Refer to the README for more detailed instruction on defining a chemical reaction network.

In [1]:
# define a reaction dictionary for reactions modeled with mass action kinetics
# a reversible reaction may be defined with a "<->" character
# the forward rate constant of a given reaction must be provided first in the ordered dictionary
reaction_dict = {
    'S + E <-> SE': {'kon': 100, 'koff': 1}, # binding of substrate to enzyme
    'SE -> P + E': {'kchem': 100} # chemical step
}

reaction_dict = {
    'S + E <-> SE': {'model': 'mass-action', 'rate-constants': [100 ,1], 'rate-constant-names': ['kon', 'koff']}, # binding of substrate to enzyme
    'SE -> P + E': {'model': 'mass-action', 'rate-constants': 100, 'rate-constant-names': 'kchem'} # chemical step
}

# define an initial state for the system
# if an initial value for a specie is not give, the code assigns it as zero
initial_values = {'S': 1000, 'E': 0.1}

# define a plotting dictionary
plot_kwargs = {'title': 'Kinetic Simulation',
                'fontsize': 12,
                'figsize': (12,12),
                'multithread': False, # turn on multithreading to leverage parallel processing
                'tol': 1e-8, # tolerance for the numerical integrator
                'S': {'min': 0, 'max': 1000, 'start': 1000, 'stepsize': 10}, # define a slider for the initial concentration of a specie
                'kchem': {'min': 0, 'max': 200, 'start': 100, 'stepsize': 1}, # define a slider for a specific rate constant
                'kon': {'min': 0, 'max': 100, 'start': 100, 'stepsize': 1}
                }

## Launching the Interactive Dashboard

In [2]:
import sys
sys.path.insert(0, '../')
from kinetics_simulator.guis import ProgressCurveGUI
from kinetics_simulator.chemicalkinetics import ChemicalReactionNetwork

reaction_network = ChemicalReactionNetwork(initial_values, reaction_dict)
figure = ProgressCurveGUI(reaction_network, plot_kwargs)
figure.interactive()

AttributeError: 'ProgressCurveGUI' object has no attribute 'plot_kwargs'

## Simulation Using Michaelis Menten Kinetics
The above simulations were generated by defining and integrating the mass action kinetic equations for a given chemical reaction network. However, when dealing with enzymatic reactions, it is often times more convenient to use Michaelis Menten kinetic model. This model is built from mass action kinetics with a few simplifying assumptions. Below is an example containing two coupled reactions: an enzymatic reaction (modeled with Michaelis Menten kinetics) and a small molecule association reaction.

In [9]:
# define a reaction dictionary for reactions modeled with mass action kinetics
# a reversible reaction may be defined with a "<->" character
# the forward rate constant of a given reaction must be provided first in the ordered dictionary
mass_action_dict = {
    'P + A <-> PA': OrderedDict({'kon': 100, 'koff': 1}), 
}

# define a dictionary for reactions modeled with Michaelis Menten kinetics
michaelis_menten_dict = {
    'S + E -> P + E': OrderedDict({'Km': 100, 'kcat': 10})
}


mass_action_dict = {
    'P + A <-> PA': {'model': 'michaelis-menten', 'rate-constants': }, 
}

# define a dictionary for reactions modeled with Michaelis Menten kinetics
michaelis_menten_dict = {
    'S + E -> P + E': OrderedDict({'Km': 100, 'kcat': 10})
}

# define an initial state for the system
# if an initial value for a specie is not give, the code assigns it as zero
initial_values = {'S': 1000, 'E': 0.1, 'A': 100}

# define a plotting dictionary
plot_kwargs = {'title': 'Kinetic Simulation',
                'fontsize': 12,
                'figsize': (12,12),
                'multithread': False, # turn on multithreading to leverage parallel processing
                'tol': 1e-8, # tolerance for the numerical integrator
                'S': {'min': 0, 'max': 1000, 'start': 1000, 'stepsize': 10}, # define a slider for the initial concentration of a specie
                'kon': {'min': 0, 'max': 100, 'start': 100, 'stepsize': 1}
                }

# launch dashboard
reaction_network = ChemicalReactionNetwork(mass_action_dict, michaelis_menten_dict, initial_values, time, concentration_units, time_units)
figure = ProgressCurveGUI(reaction_network, plot_kwargs)
figure.interactive()

SyntaxError: invalid syntax (<ipython-input-9-e2b75055806e>, line 15)

## Fitting a Simple Kinetic Model
Below is an exmaple of how one can go about fitting a kinetic model using some methods inherent to the ChemicalReactionNetwork class. The fitting algorithm requires the following user-provided inputs: a kinetic model, a set of initial concentrations for each species, and progress curve data for each initial concentration. With the required inputs, 

In [10]:
import numpy as np

reaction_dictionary = {
    'A + B <-> C': {'model': 'mass-action', 'rate-constants': [0.001,0.002], 'rate-constant-names': ['k1', 'k-1']},
}
plot_kwargs = {'title': 'Test',
                'fontsize': 12,
                'figsize': (12,12),
                'multithread': True, # turn on multithreading to leverage parallel processing
                'tol': 1e-8, # tolerance for the numerical integrator
                'A': {'min': 0, 'max': 1000, 'start': 100, 'stepsize': 1}
                }

initial_values = {'A': 100, 'B': 1000}
rxn = ChemicalReactionNetwork(initial_values, reaction_dictionary, time=np.linspace(0,30,500))
# result = rxn.fit(variable='A', observable='C', params=['k1', 'k-1'], fitting_concentrations=np.array([10, 50, 100, 500, 1000]))

# # the value of k1 and k-1 respectively
# print(result.x)

In [11]:
figure = ProgressCurveGUI(rxn, plot_kwargs)
figure.interactive()

    'data': [{'name': 'B',
              'type': 'scatter',
              'uid':…

## Fitting a Kinetic Model with Input Data
Below is an example of how one can fit a kinetic model with input experimental data. For this illustrative example, we will be using Purine Nucleoside Phosphorylase kinetic data.

In [7]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '../')
from kinetics_simulator.guis import ProgressCurveGUI
from kinetics_simulator.chemicalkinetics import ChemicalReactionNetwork

reaction_dictionary = {
    'PNP + Pi <-> PNP:Pi': {'model': 'mass-action', 'rate-constants': [0.001,0.002], 'rate-constant-names': ['kon', 'koff']},
    'PNP:Pi -> PNP + X': {'model': 'mass-action', 'rate-constants': 0.002, 'rate-constant-names': 'kcat'}
}

data = pd.read_csv('./PNP_data.csv')
Pi_concens = [float(concen.split(' ')[0]) for concen in data.columns[1:-1]]
time = data['time'].to_numpy()
timecourse = data.iloc[:, 1:-1].to_numpy().T


initial_values = {'PNP': 10e-3}
rxn = ChemicalReactionNetwork(initial_values, reaction_dictionary, time=time)
result = rxn.fit(variable='Pi', observable='X', params=['kon', 'koff', 'kcat'], fitting_concentrations=Pi_concens, ground_truth_data=timecourse)

# # the value of k1 and k-1 respectively
# print(result.x)

array([1.e-50, 1.e-50, 1.e-02, 1.e-50])